<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np

np.random.seed(812)

import pandas as pd

datafilename = 'WA_Fn-UseC_-Telco-Customer-Churn.csv'
# Load data
df = pd.read_csv(datafilename)
print(df.shape)


(7043, 21)


In [2]:
df = df.replace(" ", np.NaN).dropna()
print(df.shape)

df_bak = df.copy()


(7032, 21)


In [3]:
def is_male_bool(val):
    if (val == 'Male'):
        return 0
    else: 
        return 1

def zero_one_to_0_1(val):
    if(val == 0): 
        return 0
    else: 
        return 1
    
def yes_no_to_0_1(val):
    if(val == 'No'): 
        return 0
    else: 
        return 1
    
def n_y_NIS_to_int(val):
    if (val == 'No'):
        return 0
    elif (val == 'Yes'):
        return 2
    elif (val ==  'No internet service'):
        return 1
    else:
        return "Error"
    
def phone_service_to_int(val):
    if (val == 'No'):
        return 0
    elif (val == 'Yes'):
        return 2
    elif (val ==  'No phone service'):
        return 1
    else:
        return "Error"
    
def Internet_service_to_int(val):
    if (val == 'No'):
        return 0
    elif (val == 'DSL'):
        return 1
    elif (val ==  'Fiber optic'):
        return 2
    else:
        return "Error"
    
def Contract_to_Int(val):
    if (val == 'Month-to-month'):
        return 0
    elif (val == 'One year'):
        return 1
    elif (val ==  'Two year'):
        return 2
    else:
        return "Error"

    
def PaymentMethod(val):
    if (val == 'Electronic check'):
        return 0
    elif (val == 'Mailed check'):
        return 1
    elif (val == 'Bank transfer (automatic)'):
        return 2
    elif (val == 'Credit card (automatic)'):
        return 3 
    else:
        return "Error" 
    

In [4]:
df = df_bak.copy()
def munge(df):
    df.replace(" ", np.NaN).dropna(inplace=True)

    df['male'] = df['gender'].apply(is_male_bool)
    
    df['SeniorCitizen'] = df['SeniorCitizen'].apply(zero_one_to_0_1)
    
    for col in ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']:
        df[col] = df[col].apply(yes_no_to_0_1)
    
    for col in ['DeviceProtection', 'OnlineSecurity', 'OnlineBackup','TechSupport', 
                'StreamingTV', 'StreamingMovies']:
        df[col] = df[col].apply(n_y_NIS_to_int)
        
    for col in ['MultipleLines']:
        df[col] = df[col].apply(phone_service_to_int)
        
    for col in ['InternetService']:
        df[col] = df[col].apply(Internet_service_to_int)
        
    for col in ['Contract']:
        df[col] = df[col].apply(Contract_to_Int)
        
    for col in ['PaymentMethod']:
        df[col] = df[col].apply(PaymentMethod)
        
    df['TotalCharges'] = df['TotalCharges'].astype(float)
        
    df.drop(columns=[
                        'customerID', 
                    'gender'
# #                      'gender', 'SeniorCitizen', 'Partner', 'Dependents'
                        ], inplace=True) # unique, so no information
    return df


In [5]:
df = df_bak.copy()

target = 'Churn'

features = list ( set(df.columns) - {target} )

input_node_count = len(features)

X = df[features]
y = df[target]

X_munged = munge(X)

X_munged.dtypes
# X_munged.head()
# df.MultipleLines

OnlineBackup          int64
MonthlyCharges      float64
DeviceProtection      int64
PaperlessBilling      int64
StreamingMovies       int64
MultipleLines         int64
SeniorCitizen         int64
PhoneService          int64
StreamingTV           int64
InternetService       int64
TechSupport           int64
OnlineSecurity        int64
TotalCharges        float64
tenure                int64
Contract              int64
Partner               int64
Dependents            int64
PaymentMethod         int64
male                  int64
dtype: object

In [6]:
y = y.apply(lambda x: 1 if x == 'Yes' else 0)
y

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7032, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(X_munged)
y_train, y_test = train_test_split(y)

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# print(X_train_scaled[:10])

In [9]:
# Important Hyperparameters
inputs = X_train_scaled.shape[1]

epochs = 50
batch_size = 10

model = Sequential()
model.add(Dense(input_node_count, input_dim=inputs, kernel_initializer='normal', activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                400       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1344      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 5,969
Trainable params: 5,969
Non-trainable params: 0
_________________________________________________________________
Train on 5274 samples, validate on 1758 samples
Epoch 1/50
5274/5274 [==============================] - 25s 5ms/sample - loss: 2.5854 - mse: 2.5854 - mae: 0.5976 - val_loss: 0.2372 - val_mse: 0.2372 - val_mae: 0.3540
Epoch 2/50
5274/5274 [=============

- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?

In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# New model

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?